In [ ]:
import pandas as pd
import collections
from mylibs.mylibs import grid_search

# Parameter control panel

In [ ]:
start = '2020-01-01 08:00:00'                                                         # the start of the bactesting period 
end = None                                                                            # the end of the backtesting period
initial_cash = 10000000.0                                                             # cash invested

parameter_dic = collections.OrderedDict()
parameter_dic['factor_span']      = [1.5, 2, 2.5, 3, 3.5, 4]                          # control the width of two bands
parameter_dic['atr_period_span']  = [6, 7, 8, 9, 10, 12, 14, 16, 20]                  # use how many days to calculate ATR
parameter_dic['order_size_span']  = [50, 60, 70, 75, 80, 85, 90, 100]                 # percentage of cash for each order
parameter_dic['dema_length_span'] = [30, 60, 100, 120, 144, 169, 252, 576, 676]       # the lookback period of dema 

### We can not change these parameters in real life

In [ ]:
commission = 0.0003                                                                   # commission charged by exchange
slippage   = 0.0002                                                                   # slippage

***

# Grid search

In [ ]:
data = pd.read_csv(f'data/btcusdt_4h.csv')
data.set_index('time', inplace=True)

In [ ]:
%%time
grid_search_result = grid_search(data, start, end, initial_cash, parameter_dic, commission, slippage)

In [ ]:
grid_search_result.to_csv(f'results/grid_search_result_binance.csv', index=0)

***

# Choose parameters

<p>We use the first 4 time period to choose parameters and use the last 1 time period to test.</p>
<p>The criteria is that: calculate the Calmar Ratio of the first 4 periods, then find a threshold value to filter records according to Calmar Ratio, the Calmar Ratio of four periods should all be higher than that threshold, we stop when there is only one records left, finally, we use the parameters of that record.</p>

In [ ]:
grid_search_result = pd.read_csv('results/grid_search_result_binance.csv')

In [ ]:
def threhold_of(variable):  
    """use the mean value of Calmar Ratio of all four periods to do the first seperation"""
    sum_ = 0
    for i in range(4):
        sum_ += grid_search_result[f'{i}_{variable}'].mean()
    threshold = sum_ / 4

    condition = (grid_search_result[f'0_{variable}'] > threshold)
    for i in range(1, 4):
        condition *= (grid_search_result[f'{i}_{variable}'] > threshold)
        
    '''use count to prevent deathloop'''
    count = 0
    
    while len(grid_search_result[condition]) != 1:
        # when there are more than one records left, raise threshold
        if len(grid_search_result[condition]) > 1:
            threshold += 0.01
        # when there is no records left, decrease threshold
        else:
            threshold -= 0.01
            
        condition = (grid_search_result[f'0_{variable}'] > threshold)
        for i in range(1, 4):
            condition *= (grid_search_result[f'{i}_{variable}'] > threshold)
            
        count += 1
        if count >= 1000 and len(grid_search_result[condition]) >= 1:
            return threshold
    else:
        return threshold

In [ ]:
variable = 'calmar'
threshold = threhold_of(variable)
condition = generate_condition(variable, threshold)
grid_search_result[condition]

Then we use parameter dict {'factor': 3.5, 'atr_period': 6, 'order_size': 50, 'dema_length': 252}